In [1]:

# for loading/processing the images  
from tensorflow.keras.preprocessing.image import load_img 
from tensorflow.keras.preprocessing.image import img_to_array 
from tensorflow.keras.applications.vgg16 import preprocess_input 

# models 
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.models import Model

# clustering and dimension reduction
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# for everything else
import os
import numpy as np
import matplotlib.pyplot as plt
from random import randint
import pandas as pd
import pickle

In [2]:
path = os.getcwd() + "\..\Dataset"
# this list holds all the image filename
pottery = []

# Define a function to recursively walk through directories
def collect_images(directory):
    # Use os.scandir() to list files in the current directory
    with os.scandir(directory) as entries:
        for entry in entries:
            # If it's a directory, recurse into it
            if entry.is_dir():
                collect_images(entry.path)
            # If it's a file, check if it's an image file
            elif entry.is_file() and entry.name.lower().endswith(('.jpg', '.jpeg', '.png', '.gif', '.bmp')):
                # Add the image file to the list
                pottery.append(entry.path)

collect_images(path)

print(pottery[:10])
n_images = len(pottery)
print(f"There are {n_images} images in the dataset")

['c:\\Users\\brayk\\Documents\\CMU Classes\\interp\\Contribution\\Code\\..\\Dataset\\Banpo\\Banpo_1.jpg', 'c:\\Users\\brayk\\Documents\\CMU Classes\\interp\\Contribution\\Code\\..\\Dataset\\Banpo\\Banpo_10.jpg', 'c:\\Users\\brayk\\Documents\\CMU Classes\\interp\\Contribution\\Code\\..\\Dataset\\Banpo\\Banpo_100.jpg', 'c:\\Users\\brayk\\Documents\\CMU Classes\\interp\\Contribution\\Code\\..\\Dataset\\Banpo\\Banpo_101.jpg', 'c:\\Users\\brayk\\Documents\\CMU Classes\\interp\\Contribution\\Code\\..\\Dataset\\Banpo\\Banpo_102.jpg', 'c:\\Users\\brayk\\Documents\\CMU Classes\\interp\\Contribution\\Code\\..\\Dataset\\Banpo\\Banpo_103.jpg', 'c:\\Users\\brayk\\Documents\\CMU Classes\\interp\\Contribution\\Code\\..\\Dataset\\Banpo\\Banpo_104.jpg', 'c:\\Users\\brayk\\Documents\\CMU Classes\\interp\\Contribution\\Code\\..\\Dataset\\Banpo\\Banpo_105.jpg', 'c:\\Users\\brayk\\Documents\\CMU Classes\\interp\\Contribution\\Code\\..\\Dataset\\Banpo\\Banpo_106.jpg', 'c:\\Users\\brayk\\Documents\\CMU Class

In [ ]:
model = VGG19()
model = Model(inputs = model.inputs, outputs = model.layers[-2].output)
def extract_features(file, model):
    # load the image as a 224x224 array
    img = load_img(file, target_size=(224,224))
    # convert from 'PIL.Image.Image' to numpy array
    img = np.array(img) 
    # reshape the data for the model reshape(num_of_samples, dim 1, dim 2, channels)
    reshaped_img = img.reshape(1,224,224,3) 
    # prepare image for model
    imgx = preprocess_input(reshaped_img)
    # get the feature vector
    features = model.predict(imgx, verbose=False)
    return features

Get feature data from these images by passing them through the model (may take some time)

In [ ]:
data = {}
q = os.getcwd() + "\..\AnalData_location"
# loop through each image in the dataset
for idx,pot in enumerate(pottery):
    if idx%100 == 0:
        print(f"{idx}/{n_images}")
    # try to extract the features and update the dictionary
    feat = extract_features(pot,model)
    data[pot] = feat
          
 
# get a list of the filenames
filenames = np.array(list(data.keys()))

# get a list of just the features
feat = np.array(list(data.values()))
print(feat.shape)
feat = feat.reshape(-1,4096)
print(feat.shape)
with open(q,'wb') as file:
    pickle.dump(data,file)

Load feature data from AnalData_location (quick but requires this data to already exist)

In [3]:
q = os.getcwd() + "\..\AnalData_location"

with open(q,'rb') as file:
    data = pickle.load(file)
# get a list of the filenames
filenames = np.array(list(data.keys()))

# get a list of just the features
feat = np.array(list(data.values()))
print(feat.shape)
feat = feat.reshape(-1,4096)

(1106, 1, 4096)


In [4]:
clusters = []
cluster_types = {"Banpo":0,"Banshan":1,"Machang":2,"Majiayao":3,"Miaodigou":4}
for path in filenames:
    filename = os.path.basename(path)
    name = filename[:filename.index("_")]
    clusters.append(cluster_types[name])

In [5]:
n_components = 10
pca = PCA(n_components=n_components, random_state=76101)
pca.fit(feat)
x = pca.transform(feat)
print(f"Components before PCA: {feat.shape[1]}")
print(f"Components after PCA: {pca.n_components}")

Components before PCA: 4096
Components after PCA: 10


In [6]:
from sklearn.metrics import silhouette_score


In [7]:
score = silhouette_score(x, clusters)
print(score)

0.05433478
